In [67]:
import pandas as pd
import numpy as np
import functions
from sklearn.preprocessing import StandardScaler

In [25]:
from indicators import *

In [4]:
df = pd.read_csv("coinbase_petit.csv")

In [14]:
df.columns = [v.upper() for v in list(df.columns)]

In [12]:
list(df.columns[2:6])

['Open', 'High', 'Low', 'Close']

In [13]:
[v.upper() for v in list(df.columns[2:6])]

['OPEN', 'HIGH', 'LOW', 'CLOSE']

In [20]:
df = df.rename(columns={df.columns[6]: 'VOLUME'})

In [29]:
ema(df, period=7, column='CLOSE')

,UNNAMED: 0,TIMESTAMP,OPEN,HIGH,LOW,CLOSE,VOLUME,VOLUME_.CURRENCY.,WEIGHTED_PRICE,ema0,ema7
500,501,1417441980,370,370,370,370,0.026556,9.82555,370,370.0,370.0
501,502,1417442040,370,370,370,370,0.026556,9.82555,370,370.0,370.0
502,503,1417442100,370,370,370,370,0.026556,9.82555,370,370.0,370.0
503,504,1417442160,370,370,370,370,0.026556,9.82555,370,370.0,370.0
504,505,1417442220,370,370,370,370,0.026556,9.82555,370,370.0,370.0
505,506,1417442280,370,370,370,370,0.026556,9.82555,370,370.0,370.0
506,507,1417442340,370,370,370,370,0.026556,9.82555,370,370.0,370.0
507,508,1417442400,370,370,370,370,0.026556,9.82555,370,370.0,370.0
508,509,1417442460,370,370,370,370,0.026556,9.82555,370,370.0,370.0
509,510,1417442520,370,370,370,370,0.026556,9.82555,370,370.0,370.0


In [59]:
pre_df = functions.data_load_partial("new_data_btc_small.csv", 40000)
    # range included in case you don't want to load the whole file (for testing
    # and memory saving purposes)
df = functions.filter_coins(pre_df)

In [80]:
def compute_one_rsi(df):

    ups = df[df > 0].mean(skipna=True, axis = 1)
    downs = df[df < 0].mean(skipna=True, axis = 1)
    rs = ups/(-1*downs)
    rs.fillna(0, inplace = True)
    rsi = rs/(1+rs)

    return rsi

def compute_normalized_rsi(df, forecast_length, periods = 7):
    df_h = df[df.columns[::forecast_length]].iloc[::2]
    after = df_h.iloc[:,1:]
    before = df_h.iloc[:,:-1]
    diffs = (after - before.values)/before.values

    rsi = pd.DataFrame(index = diffs.index)

    for index, column in enumerate(diffs):
        rsi[column] = pd.Series(compute_one_rsi(diffs.iloc[:,index:(index + periods*24)]), index = rsi.index)

    rsi.index = [x.replace("p","rsi") for x in rsi.index]
    
    rsi_transpose = rsi.transpose() # need to transpose because StandardScaler only works on rows
    rsi_scaler = StandardScaler().fit(rsi_transpose) # define the scaler
    rsi_transpose_std = rsi_scaler.transform(rsi_transpose) # rescale data
    rsi_normalized = rsi_transpose_std.transpose() # retranspose back to original
    rsi_normalized = pd.DataFrame(data=rsi_normalized,    # values
              index=rsi.index,    # 1st column as index
              columns=diffs.iloc[:,forecast_length:].columns) 

    return rsi_normalized, rsi_scaler

In [60]:
forecast_length = 60
df_h = df[df.columns[::forecast_length]].iloc[::2]

In [61]:
after = df_h.iloc[:,1:]
before = df_h.iloc[:,:-1]
diffs = (after - before.values)/before.values

In [77]:
rsi_normalized, rsi_scaler = compute_normalized_rsi(df, 60, 7)

In [76]:
import importlib
importlib.reload(functions)

<module 'functions' from 'C:\\Users\\eudald\\Desktop\\Crypto-master\\Crypto\\src\\AI\\functions.py'>

In [78]:
rsi_normalized.shape

(120, 273)

In [ ]:
X_train.shape